In [ ]:
import neurokit2 as nk
import mne
import os
import os.path as op
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt
import re


In [ ]:
raw = mne.io.read_raw('D:/hse/psychodelic_like_experience/data_processing/eeg_cleaned_ica/FX48.fif')

In [ ]:
events = mne.read_events('D:/hse/psychodelic_like_experience/data_processing/events/fx48_eve.txt')

In [ ]:
raw.first_samp

In [ ]:
all_events_id = {
    'FractalBlue': 101,
    'FractalPurple': 102,
    'FractalGreen': 103,
    'FractalRed': 104,
    'FractalYellow': 105,

    'kaleidoscopeBlue': 201,
    'kaleidoscopePurple': 202,
    'kaleidoscopeGreen': 203,
    'kaleidoscopeRed': 204,
    'kaleidoscopeYellow': 205,

    'CubesControlPinkBlue': 301,
    'CubesControlPurpleGreen': 302,
    'CubesControlBlueRed': 303,
    'CubesControlPinkYellow': 304,
    'CubesControlGreenOrange': 305,

    'HoneyCombBluePink': 401,
    'HoneyCombGreenPurple': 402,
    'HoneyCombRedBlue': 403,
    'HoneyCombOrangePink': 404,
    'HoneyCombGreenOrange': 405
}

event_id = {
    'HoneyCombGreenPurple': 0,
    'FractalGreen': 1,
    'FractalYellow': 2,
    'CubesControlPinkYellow': 3,
    'kaleidoscopePurple': 4,
    'CubesControlBlueRed': 5,
    'HoneyCombOrangePink': 6,
    'CubesControlPurpleGreen': 7,
    'FractalPurple': 8,
    'FractalRed': 9,
    'HoneyCombRedBlue': 10,
    'CubesControlPinkBlue': 11,
    'kaleidoscopeYellow': 12,
    'kaleidoscopeGreen': 13,
    'FractalBlue': 14,
    'CubesControlGreenOrange': 15,
    'kaleidoscopeBlue': 16,
    'HoneyCombGreenOrange': 17,
    'kaleidoscopeRed': 18,
    'HoneyCombBluePink': 19
}

# Create a new dictionary by mapping labels to numbers
EVENT_ID = {label: all_events_id.get(label, None) for label in event_id}

# Print the merged dictionary
print(EVENT_ID)


In [ ]:
def replace_substrings(key):
    # Define the replacement rules
    replacements = {"Fractal": "Fractal/","Control": "Control/", "HoneyComb":"HoneyComb/", "kaleidoscope": "kaleidoscope/"}  # Add more as needed

    # Apply replacements
    for old, new in replacements.items():
        key = key.replace(old, new)

    return key


In [ ]:
new_dict = {}

for key, value in EVENT_ID.items():
    new_key = replace_substrings(key)
    new_dict[new_key] = value

print(new_dict)


In [ ]:
epochs = mne.Epochs(raw, events, new_dict, tmin=-5, tmax=60, baseline=(-5, 0), preload = True)

In [ ]:
epochs.plot( n_epochs=1, n_channels=20, title=None, show=True, block=False, decim='auto')

In [ ]:
#psd

In [ ]:
epo_spectrum = epochs.compute_psd()
psds, freqs = epo_spectrum.get_data(return_freqs=True)
print(f"\nPSDs shape: {psds.shape}, freqs shape: {freqs.shape}")
epo_spectrum

In [ ]:
evoked = epochs["Fractal"].average()
evk_spectrum = evoked.compute_psd()
# the first 3 frequency bins for the first 4 channels:
evk_spectrum.plot_topomap(ch_type="eeg", agg_fun=np.median)


In [ ]:
evoked = epochs["kaleidoscope"].average()
evk_spectrum = evoked.compute_psd()
# the first 3 frequency bins for the first 4 channels:
evk_spectrum.plot_topomap(ch_type="eeg", agg_fun=np.median)


In [ ]:
evoked = epochs["HoneyComb"].average()
evk_spectrum = evoked.compute_psd()
# the first 3 frequency bins for the first 4 channels:
evk_spectrum.plot_topomap(ch_type="eeg", agg_fun=np.median)


In [ ]:
evoked = epochs["CubesControl"].average()
evk_spectrum = evoked.compute_psd()
# the first 3 frequency bins for the first 4 channels:
evk_spectrum.plot_topomap(ch_type="eeg", agg_fun=np.median)

In [ ]:
#time freq

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import mne
from mne.datasets import somato
from mne.time_frequency import tfr_morlet

In [ ]:
freqs = np.logspace(*np.log10([6, 35]), num=8)
n_cycles = freqs / 2.0  # different number of cycle per frequency
power, itc = tfr_morlet(
    epochs["Fractal"],
    freqs=freqs,
    n_cycles=n_cycles,
    use_fft=True,
    return_itc=True,
    decim=3,
    n_jobs=None)


In [ ]:
raw.ch_names

In [ ]:
power.plot_topo(baseline=None, title="Average power")
power.plot([10], baseline=None, title=power.ch_names[10])

fig, axes = plt.subplots(1, 3, figsize=(7, 4), layout="constrained")
topomap_kw = dict(
    ch_type="eeg", tmin=2, tmax=4, baseline=None, show=False
)
plot_dict = dict(Theta = dict(fmin=4, fmax=8), Alpha=dict(fmin=8, fmax=12),  Beta=dict(fmin=13, fmax=25))
for ax, (title, fmin_fmax) in zip(axes, plot_dict.items()):
    power.plot_topomap(**fmin_fmax, axes=ax, **topomap_kw)
    ax.set_title(title)

In [ ]:
#connectivity

In [ ]:
from mne_connectivity import spectral_connectivity_epochs
from mne_connectivity.viz import plot_sensors_connectivity

In [ ]:
fmin, fmax = 0, 5
sfreq = raw.info["sfreq"]  # the sampling frequency
tmin = 0.0  # exclude the baseline period
epochs.load_data().pick("eeg") 
con = spectral_connectivity_epochs(
    epochs["Fractal"],
    method="pli",
    mode="multitaper",
    sfreq=sfreq,
    fmin=fmin,
    fmax=fmax,
    faverage=True,
    tmin=tmin,
    mt_adaptive=False,
    n_jobs=1,
)

# Now, visualize the connectivity in 3D:
plot_sensors_connectivity(epochs.info, con.get_data(output="dense")[:, :, 0])

In [ ]:
fmin, fmax = 0, 5
sfreq = raw.info["sfreq"]  # the sampling frequency
tmin = 0.0  # exclude the baseline period
epochs.load_data().pick("eeg") 
con = spectral_connectivity_epochs(
    epochs["kaleidoscope"],
    method="pli",
    mode="multitaper",
    sfreq=sfreq,
    fmin=fmin,
    fmax=fmax,
    faverage=True,
    tmin=tmin,
    mt_adaptive=False,
    n_jobs=1,
)

# Now, visualize the connectivity in 3D:
plot_sensors_connectivity(epochs.info, con.get_data(output="dense")[:, :, 0])

In [ ]:
fmin, fmax = 0, 5
sfreq = raw.info["sfreq"]  # the sampling frequency
tmin = 0.0  # exclude the baseline period
epochs.load_data().pick("eeg") 
con = spectral_connectivity_epochs(
    epochs["HoneyComb"],
    method="pli",
    mode="multitaper",
    sfreq=sfreq,
    fmin=fmin,
    fmax=fmax,
    faverage=True,
    tmin=tmin,
    mt_adaptive=False,
    n_jobs=1,
)

# Now, visualize the connectivity in 3D:
plot_sensors_connectivity(epochs.info, con.get_data(output="dense")[:, :, 0])

In [ ]:
fmin, fmax = 0, 5
sfreq = raw.info["sfreq"]  # the sampling frequency
tmin = 0.0  # exclude the baseline period
epochs.load_data().pick("eeg") 
con = spectral_connectivity_epochs(
    epochs["CubesControl"],
    method="pli",
    mode="multitaper",
    sfreq=sfreq,
    fmin=fmin,
    fmax=fmax,
    faverage=True,
    tmin=tmin,
    mt_adaptive=False,
    n_jobs=1,
)

# Now, visualize the connectivity in 3D:
plot_sensors_connectivity(epochs.info, con.get_data(output="dense")[:, :, 0])